In [ ]:
from dotenv import load_dotenv
import os
import re
import os
import pandas as pd
import numpy as np
from groq import Groq
from stockfish import Stockfish

In [16]:
load_dotenv()  # Load variables from .env
STOCKFISH_PATH=os.environ.get("STOCKFISH_PATH")
LLAMA_MODEL_PATH=os.environ.get("LLAMA_MODEL_PATH")
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
stockfish = Stockfish(STOCKFISH_PATH,depth=25,parameters={"Threads": 20,"Minimum Thinking Time": 10})
d1=pd.read_csv('theory/a.tsv', sep='\t', header=0)
d2=pd.read_csv('theory/b.tsv', sep='\t', header=0)
d3=pd.read_csv('theory/c.tsv', sep='\t', header=0)
d4=pd.read_csv('theory/d.tsv', sep='\t', header=0)
d5=pd.read_csv('theory/e.tsv', sep='\t', header=0)

In [ ]:
def clean(response):
    response=re.sub(r'<think>.*?</think>','',response,flags=re.DOTALL)
    return response

In [ ]:
def get_best_move(fen):
    stockfish.set_fen_position(fen)
    return stockfish.get_best_move()

In [ ]:
def ch_comp_bm_w_exp(fen,best_move,type_of_move,evaluation):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a top-tier chess coach with deep strategic and tactical mastery. Your task is to recommend the best move with absolute clarity, using proper chess notation and piece names. Justify the move with precise reasoning—covering positional, tactical, and strategic factors. Highlight threats, weaknesses, and long-term plans. Keep explanations concise, potent, and highly structured to maximize the user's chess understanding.",
            },
            {
                "role": "user",
                "content": f"Justify why {type_of_move} {best_move} is the best move for the FEN {fen} with an evaluation of {evaluation}",
            }
        ],
        model="deepseek-r1-distill-llama-70b",
    )

    return clean(chat_completion.choices[0].message.content)

In [ ]:
def ch_comp_th(eco,name,pgn):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"You are a chess coach , explain a theory behind the opening of the move with the name {name} ,ECO {eco} and the PGN {pgn} , please tell ALL THE pros and cons of the opening . All the variations and how to play against them.",
            },
            {
                "role": "user",
                "content": f"Explain the pros and cons and how to play against the best variations of the opening of the name {name} ,ECO {eco} and the PGN {pgn}",
            }
        ],
        model="deepseek-r1-distill-llama-70b",
    )

    return clean(chat_completion.choices[0].message.content)

In [ ]:
def find(theory):
    for x in range(0,len(d1["name"])):
        if d1["name"][x].lower()==theory.lower():
            return d1['eco'][x],d1['name'][x],d1['pgn'][x]
    for x in range(0,len(d2["name"])):
        if d2["name"][x].lower()==theory.lower():
            return d2['eco'][x],d2['name'][x],d2['pgn'][x]
    for x in range(0,len(d3["name"])):
        if d3["name"][x].lower()==theory.lower():
            return d3['eco'][x],d3['name'][x],d3['pgn'][x]
    for x in range(0,len(d4["name"])):
        if d4["name"][x].lower()==theory.lower():
            return d4['eco'][x],d4['name'][x],d4['pgn'][x]
    for x in range(0,len(d5["name"])):
        if d5["name"][x].lower()==theory.lower():
            return d5['eco'][x],d5['name'][x],d5['pgn'][x]

In [ ]:
def th_w_exp():
    theory=input("You : ")
    eco,name,pgn=find(theory)
    return ch_comp_th(eco,name,pgn)

In [ ]:
from math import ceil

def get_eval(fen):
    stockfish.set_fen_position(fen)
    x=stockfish.get_evaluation()
    if x["type"]=="mate":
        return f"Mate in {x['value']}"
    else:
        return f"{round((int(x['value'])**0.5) / 10.0, 1)} pawn advantage for {'white' if x['value'] > 0 else 'black'}"

In [ ]:
def bm_w_exp():
    fen = input("You : ")
    best_move = get_best_move(fen)
    g = str(stockfish.will_move_be_a_capture(best_move)).split(".")[1]
    if g=="NO_CAPTURE":
        g=False
    else:
        g=True
    evaluation=get_eval(fen)
    p1_split = str(stockfish.get_what_is_on_square(best_move[:2])).split(".")
    p2_split = str(stockfish.get_what_is_on_square(best_move[2:])).split(".")
    p1 = p1_split[1] if len(p1_split) > 1 else p1_split[0]
    p2 = p2_split[1] if len(p2_split) > 1 else p2_split[0]
    if g:
        type_of_move = f"Capturing the {p2} with {p1}"
    elif "e8g8" in best_move and "KING" in p1:
        type_of_move = f"Castling with the {p1}"
    elif "e8b8" in best_move and "KING" in p1:
        type_of_move = f"Castling with the {p1}"
    elif "e1g1" in best_move and "KING" in p1:
        type_of_move = f"Castling with the {p1}"
    elif "e1b1" in best_move and "KING" in p1:
        type_of_move = f"Castling with the {p1}"
    else:
        type_of_move = "Move"
    print("Bot : ", ch_comp_bm_w_exp(fen, best_move, type_of_move,evaluation))

In [19]:
import streamlit as st
from streamlit_chat import message

message("My message") 
message("Hello bot!", is_user=True)  # align's the message to the right

2025-02-18 19:21:12.501 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 19:21:12.553 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 19:21:12.553 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 19:21:12.553 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 19:21:12.588 
  command:

    streamlit run /opt/anaconda3/envs/BasicChatgpt/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-02-18 19:21:12.589 Thread 'MainThread': missing ScriptRunContext! This warning ca